# Install Hakai API client

In [1]:
%pip install hakai_api
%pip install gsw

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import Packages

In [2]:
from hakai_api import Client
import pandas as pd
import gsw

## Get Credentials From Hakai
This step creates an Hakai API connection, retrieve your Hakai account information and creates a temporary token saved locally in your temporary files to be able to recognize you and give you access to Hakai's data.

In [3]:
client = Client()

## Download Data
This section download some data from the Hakai database, through the Hakai API. For a more information about the Hakai API, see https://hakaiinstitute.github.io/hakai-api/

In [5]:
# This endpoint retrieve the hakai ctd data 
# (see here for more endpoints https://hakaiinstitute.github.io/hakai-api/endpoints/)
endpoint = "ctd/views/file/cast/data" 

# This section of the url will filter the ctd data on the database 
# and provide use only the data matchingg those conditions:
# (see documentation here https://hakaiinstitute.github.io/hakai-api/querying_data/)
filter_endpoint = (
    "station=QU39" # at station QU39
    + "&start_dt>2020-01-01&start_dt<2022-01-01" # from 2020 and before 2022
    + "&direction_flag=d" # downcast data only
    + "&limit=-1" # Get all the data that respects those conditions above
)

# Generate the query and send query to Hakai API
query = f"{client.api_root}/{endpoint}?{filter_endpoint}"
print(f"Query used:\n{query}")
response = client.get(query)

# Review if the server is happy with our query
if response != response.status_code:
    # Send an error if it's not working and tell us why
    response.raise_for_status()

# Load the data in json format to a pandas dataframe called df
df = pd.DataFrame(response.json())

# Show me the first few lines of the dataframe
df.head()

Query used:
https://hecate.hakai.org/api/ctd/views/file/cast/data?station=QU39&start_dt>2020-01-01&start_dt<2022-01-01&direction_flag=d&limit=-1


,ctd_file_pk,ctd_cast_pk,hakai_id,ctd_data_pk,filename,device_model,device_sn,device_firmware,file_processing_stage,work_area,...,cdom_ppb,cdom_ppb_flag,cdom_ppb_flag_level_1,bin_stats,cast_type,sensors_submerged,water_depth_m,soak_start,soak_end,weather
0,7303,18520,65679_2021-11-29T18:11:15.000Z,10822546,065679_20211129_1259,RBRconcerto,65679,1.43,8_rbr_processed,QUADRA,...,None,None,2,None,Dynamic,Full,None,None,None,"{'tide': None, 'sky_cover': 'lifted_fog', 'win..."
1,7303,18520,65679_2021-11-29T18:11:15.000Z,10822547,065679_20211129_1259,RBRconcerto,65679,1.43,8_rbr_processed,QUADRA,...,None,None,2,None,Dynamic,Full,None,None,None,"{'tide': None, 'sky_cover': 'lifted_fog', 'win..."
2,7303,18520,65679_2021-11-29T18:11:15.000Z,10822604,065679_20211129_1259,RBRconcerto,65679,1.43,8_rbr_processed,QUADRA,...,None,None,2,None,Dynamic,Full,None,None,None,"{'tide': None, 'sky_cover': 'lifted_fog', 'win..."
3,6951,17426,1907674_2021-07-13T17:09:00Z,10606562,SBE19plus_01907674_2021_07_13_0004,SBE19plus,1907674,2.5.2,8_binAvg,QUADRA,...,None,None,2,None,Dynamic,Full,None,None,None,"{'tide': None, 'sky_cover': 'clear', 'wind_spe..."
4,7303,18520,65679_2021-11-29T18:11:15.000Z,10822556,065679_20211129_1259,RBRconcerto,65679,1.43,8_rbr_processed,QUADRA,...,None,None,2,None,Dynamic,Full,None,None,None,"{'tide': None, 'sky_cover': 'lifted_fog', 'win..."


## Generate Derived Variables
We can use the TEOS-10 standard python package (GSW Oceanographic Toolbox) to generate oceanographic variables from different measurement made by the CTD instruments.

In [6]:
# Derive Variables
df['absolute_salinity'] = gsw.SA_from_SP(df['salinity'],df['pressure'],df['longitude'],df['latitude'])
df['conservative_temperature'] = gsw.CT_from_t(df['absolute_salinity'],df['temperature'],df['pressure'])
df['sigma0'] = gsw.sigma0(df['absolute_salinity'],df['conservative_temperature'])

/Users/jessybarrette/miniconda3/lib/python3.9/site-packages/pandas/core/arraylike.py:397: RuntimeWarning: invalid value encountered in ct_from_t
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Plot Data
The graphic library **Plotly** (https://plotly.com/python/) is a great tool to generate nice interactive figures with just a few lines of code. 

The plotly express package is certainly the easiest to use: https://plotly.com/python/plotly-express/

In [7]:
import plotly.express as px

# Let's generate a scatter plot of the salinity profiles timeseries
# For mor information about he plotly express tool, see here https://plotly.com/python/line-and-scatter/
fig = px.scatter(df,x='start_dt',y='depth',color='salinity',range_color=[25,31])
fig.update_yaxes(autorange='reversed')